In [1]:
using Plots, LinearAlgebra, ProgressMeter, SparseArrays, Random, PyCall, Statistics, Distances
using FileIO, CSV, DataFrames, JLD, MLDatasets
using LightGraphs
#using Interact
plotly()

np = pyimport("numpy")
sk = pyimport("sklearn")
torch = pyimport("torch")
sk.manidold = pyimport("sklearn.manifold") 

include("EDM.jl")

gr(linewidth = 5, ytickfontsize = 12, xtickfontsize = 12, labelfontsize = 15, legendfontsize = 12)

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/rsonthal/.julia/packages/Plots/lmp2A/src/backends.jl:372


Plots.GRBackend()

In [2]:
function smooth(A, k = 7)
    B = zeros(length(A))
    n = length(A)
    for i = 1:length(B)
        count = 0
        for j = 1:k
            idx = j-Int(ceil(k/2))
            if i+idx >= 1 && i+idx <= n
                count += 1
                B[i] += A[i+idx]
            end
        end
        B[i] = B[i]/count
    end
    
    return B
end

smooth (generic function with 2 methods)

# MNIST Plots

In [45]:
dictM = load("MNIST-missing70-correct-lower.jld")
dictI = load("MNIST-isomap-correct-lower.jld")
dictP = load("MNIST-perturbed1,5-correct-lower.jld")


mds_acc_NN_P = torch.load("mds_acc-MNIST-NN-perturbed1,4-new-perturb")[:numpy]()
lower_mds_acc_NN_P = torch.load("lower_mds_acc-MNIST-NN-perturbed1,4-new-perturb")[:numpy]()
mds_acc_NN_I = torch.load("mds_acc-MNIST-NN-isomap-correct-lower")[:numpy]()
lower_mds_acc_NN_I = torch.load("lower_mds_acc-MNIST-NN-isomap-correct-lower")[:numpy]();
mds_acc_NN_M = torch.load("mds_acc-MNIST-NN-missing70-correct-lower")[:numpy]()
lower_mds_acc_NN_M = torch.load("lower_mds_acc-MNIST-NN-missing70-correct-lower")[:numpy]();

In [46]:
mds_accM = dictM["mds accuracy"][:,1]
mds_accI = dictI["mds accuracy"][:,1]
mds_accP = dictP["mds accuracy"][:,1]

lower_mds_accM = dictM["lower mds accuracy"][:,1]
lower_mds_accI = dictI["lower mds accuracy"][:,1]
lower_mds_accP = dictP["lower mds accuracy"][:,1]

true_accM = dictM["true accuracy"][:,1]
true_accP = dictP["true accuracy"][:,1]
true_accI = dictI["true accuracy"][:,1]

lower_accM = dictM["lower acuracy"][:,1]
lower_accP = dictP["lower acuracy"][:,1]
lower_accI = dictI["lower acuracy"][:,1];

In [47]:
TM = argmin(mds_accM) - 1
TP = argmin(mds_accP) - 1
TI = argmin(mds_accI) - 1

249

In [48]:
plot(collect(2:TM+1), mds_accM[1:TM], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TM+1), lower_mds_accM[1:TM], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TM+1), lower_accM[1:TM], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TM+1), true_accM[1:TM], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Missing KNN MNIST")

In [49]:
plot(collect(2:TP+1), mds_accP[1:TP], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TP+1), lower_mds_accP[1:TP], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TP+1), lower_accP[1:TP], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TP+1), true_accP[1:TP], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Perturbed KNN MNIST")

In [23]:
plot(collect(2:TI+1), mds_accI[1:TI], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TI+1), lower_mds_accI[1:TI], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TI+1), lower_accI[1:TI], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TI+1), true_accI[1:TI], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Isomap KNN MNIST")

In [51]:
plot(collect(1:1000-1),mds_acc_NN_P[2:1000], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1000-1),lower_mds_acc_NN_P[2:1000], label = "lower + cMDS", legend = :bottomleft)
savefig("Perturbed NN MNIST")

In [52]:
plot(collect(1:990-1),mds_acc_NN_I[2:990], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:990-1),lower_mds_acc_NN_I[2:990], label = "lower + cMDS", legend = :bottomleft)
savefig("Isomap NN MNIST")

In [53]:
plot(collect(1:1053-1),mds_acc_NN_M[2:end], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1053-1),lower_mds_acc_NN_M[2:end], label = "lower + cMDS", legend = :bottomleft)
savefig("Missing NN MNIST")

# Fashion MNIST Plots

In [72]:
dictM = load("FMNIST-missing70-correct-lower.jld")
dictI = load("FMNIST-isomap-correct-lower.jld")
dictP = load("FMNIST-perturbed2,1-correct-lower.jld")


mds_acc_NN_P = torch.load("mds_acc-FMNIST-NN-perturbed-new-perturb")[:numpy]()
lower_mds_acc_NN_P = torch.load("lower_mds_acc-FMNIST-NN-perturbed-new-perturb")[:numpy]()
mds_acc_NN_I = torch.load("mds_acc-FMNIST-NN-isomap-correct-lower")[:numpy]()
lower_mds_acc_NN_I = torch.load("lower_mds_acc-FMNIST-NN-isomap-correct-lower")[:numpy]();
mds_acc_NN_M = torch.load("mds_acc-FMNIST-NN-missing70-correct-lower")[:numpy]()
lower_mds_acc_NN_M = torch.load("lower_mds_acc-FMNIST-NN-missing70-correct-lower")[:numpy]();

In [73]:
mds_accM = dictM["mds accuracy"][:,1]
mds_accI = dictI["mds accuracy"][:,1]
mds_accP = dictP["mds accuracy"][:,1]

lower_mds_accM = dictM["lower mds accuracy"][:,1]
lower_mds_accI = dictI["lower mds accuracy"][:,1]
lower_mds_accP = dictP["lower mds accuracy"][:,1]

true_accM = dictM["true accuracy"][:,1]
true_accP = dictP["true accuracy"][:,1]
true_accI = dictI["true accuracy"][:,1]

lower_accM = dictM["lower acuracy"][:,1]
lower_accP = dictP["lower acuracy"][:,1]
lower_accI = dictI["lower acuracy"][:,1];

In [74]:
TM = argmin(mds_accM) - 1
TP = argmin(mds_accP) - 1
TI = argmin(mds_accI) - 1

255

In [75]:
plot(collect(2:TM+1), mds_accM[1:TM], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TM+1), lower_mds_accM[1:TM], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TM+1), lower_accM[1:TM], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TM+1), true_accM[1:TM], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Missing KNN FMNIST")

In [76]:
plot(collect(2:TP+1), mds_accP[1:TP], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TP+1), lower_mds_accP[1:TP], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TP+1), lower_accP[1:TP], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TP+1), true_accP[1:TP], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Perturbed KNN FMNIST")

In [77]:
plot(collect(2:TI+1), mds_accI[1:TI], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TI+1), lower_mds_accI[1:TI], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TI+1), lower_accI[1:TI], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TI+1), true_accI[1:TI], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Isomap KNN FMNIST")

In [79]:
plot(collect(1:1000-1),mds_acc_NN_P[2:end], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1000-1),lower_mds_acc_NN_P[2:end], label = "lower + cMDS", legend = :bottomleft)
savefig("Perturbed NN FMNIST")

In [80]:
plot(collect(1:1010-1),smooth(mds_acc_NN_I[2:end]), xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1010-1),smooth(lower_mds_acc_NN_I[2:end]), label = "lower + cMDS", legend = :bottomleft)
savefig("Isomap NN FMNIST")

In [81]:
plot(collect(1:1070-1),smooth(mds_acc_NN_M[2:end]), xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1070-1),smooth(lower_mds_acc_NN_M[2:end]), label = "lower + cMDS", legend = :bottomleft)
savefig("Missing NN FMNIST")

# Cifar 10 Plots

In [63]:
dictM = load("cifar-missing70-correct-lower.jld")
dictI = load("cifar-isomap-correct-lower.jld")
dictP = load("cifar-perturbed5,8-correct-lower.jld")


mds_acc_NN_P = torch.load("mds_acc-cifar-NN-perturbed-new-perturb")[:numpy]()
lower_mds_acc_NN_P = torch.load("lower_mds_acc-cifar-NN-pertubred-new-perturb")[:numpy]()
mds_acc_NN_I = torch.load("mds_acc-cifar-NN-isomap-correct-lower")[:numpy]()
lower_mds_acc_NN_I = torch.load("lower_mds_acc-cifar-NN-isomap-correct-lower")[:numpy]();
mds_acc_NN_M = torch.load("mds_acc-cifar-NN-missing70-correct-lower")[:numpy]()
lower_mds_acc_NN_M = torch.load("lower_mds_acc-cifar-NN-missing70-correct-lower")[:numpy]();

In [64]:
mds_accM = dictM["mds accuracy"][:,1]
mds_accI = dictI["mds accuracy"][:,1]
mds_accP = dictP["mds accuracy"][:,1]

lower_mds_accM = dictM["lower mds accuracy"][:,1]
lower_mds_accI = dictI["lower mds accuracy"][:,1]
lower_mds_accP = dictP["lower mds accuracy"][:,1]

true_accM = dictM["true accuracy"][:,1]
true_accP = dictP["true accuracy"][:,1]
true_accI = dictI["true accuracy"][:,1]

lower_accM = dictM["lower acuracy"][:,1]
lower_accP = dictP["lower acuracy"][:,1]
lower_accI = dictI["lower acuracy"][:,1];

In [65]:
TM = argmin(mds_accM) - 1
TP = argmin(mds_accP) - 1
TI = argmin(mds_accI) - 1

277

In [66]:
plot(collect(2:TM+1), mds_accM[1:TM], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TM+1), lower_mds_accM[1:TM], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TM+1), lower_accM[1:TM], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TM+1), true_accM[1:TM], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Missing KNN CIFAR")

In [67]:
plot(collect(2:TP+1), mds_accP[1:TP], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TP+1), lower_mds_accP[1:TP], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TP+1), lower_accP[1:TP], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TP+1), true_accP[1:TP], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Perturbed KNN CIFAR")

In [68]:
plot(collect(2:TI+1), mds_accI[1:TI], xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(2:TI+1), lower_mds_accI[1:TI], xlabel = "Dimension", label = "Lower + MDS")
plot!(collect(2:TI+1), lower_accI[1:TI], xlabel = "Dimension", label = "Lower")
plot!(collect(2:TI+1), true_accI[1:TI], xlabel = "Dimension", label = "True", legend = :bottomleft)
savefig("Isomap KNN CIFAR")

In [69]:
plot(collect(1:1052),smooth(mds_acc_NN_I[2:end],25), xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1052),smooth(lower_mds_acc_NN_I[2:end],25), label = "lower + cMDS", legend = :bottomleft)
savefig("Isomap NN CIFAR")

In [70]:
plot(collect(1:1000-1),smooth(mds_acc_NN_P[2:1000],7), xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1000-1),smooth(lower_mds_acc_NN_P[2:1000],7), label = "lower + cMDS", legend = :bottomleft)
savefig("Perturbed NN CIFAR")

In [71]:
plot(collect(1:1052),smooth(mds_acc_NN_M[2:end],7), xlabel = "Dimension", label = "cMDS", ylabel = "Accuracy")
plot!(collect(1:1052),smooth(lower_mds_acc_NN_M[2:end],7), label = "lower + cMDS", legend = :bottomleft)
savefig("Missing NN CIFAR")